In [ ]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta
import seaborn as sns
from scipy import stats

In [ ]:
def import_raw_data():
    """
    Imports data from the ODI-2025.csv file without modifying it, aside from column names.

    :return: The contents of ODI-2025.csv as a pandas dataframe.
    """
    usecols = ['Timestamp',
               'What programme are you in?',
               'Have you taken a course on machine learning?',
               'Have you taken a course on information retrieval?',
               'Have you taken a course on statistics?',
               'Have you taken a course on databases?',
               'What is your gender?',
               'I have used ChatGPT to help me with some of my study assignments ',
               'When is your birthday (date)?',
               'How many students do you estimate there are in the room?',
               'What is your stress level (0-100)?',
               'How many hours per week do you do sports (in whole hours)? ',
               'Give a random number',
               'Time you went to bed Yesterday',
               'What makes a good day for you (1)?',
               'What makes a good day for you (2)?']
    names = ['timestamp',
             'programme',
             'machine_learning',
             'information_retrieval',
             'statistics',
             'databases',
             'gender',
             'chatgpt',
             'birthday',
             'student_estimate',
             'stress',
             'sports',
             'random_number',
             'bedtime',
             'good_day_1',
             'good_day_2']

    df = pd.read_csv('files/ODI-2025.csv', usecols=usecols, sep=';')
    df.columns = names
    return df


def save_dataframe_to_file(df, filename):
    """
    Saves a pandas dataframe to the file DMT_Data.csv with separator ;.

    :param df: Pandas dataframe
    """
    df.to_csv('files/' + filename, index=False, sep=';')



def import_data(filename):
    return pd.read_csv('files/' + filename, sep=';')
    

def import_clean_data():
    return pd.read_csv('files/DMT_Data.csv', sep=';')

In [ ]:
def column_to_string(df, colname):
    cleaned_column = df[colname].apply(lambda x: ' '.join(x.split()))
    return ' '.join(cleaned_column)


def column_wordcloud(filename, column):
    data = pd.read_csv('files/' + filename, sep=';')
    column_string = column_to_string(data, column)
    wordcloud = WordCloud(width=800,
                          height=400,
                          background_color='white',
                          colormap='viridis'
                          ).generate(column_string)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig(f'figures/{column}_wordcloud.png', dpi=300, bbox_inches='tight')

# Data Cleaning

### Convert to Standard Formats

In [ ]:
def check_ai(programme):
    return bool(re.search(r'\b(ai|artificial|artifical|inteligence|intelligence)\b', programme, re.IGNORECASE))


def check_cs(programme):
    return bool(re.search(r'\b(cs|computer science|comp sci)\b', programme, re.IGNORECASE))


def check_cls(programme):
    return bool(re.search(r'\b(cls|computational science)\b', programme, re.IGNORECASE))


def check_econometrics(programme):
    return bool(re.search(r'\b(econometrics|eor)\b', programme, re.IGNORECASE))


def check_finance(programme):
    return bool(re.search(r'\b(finance|fintech)\b', programme, re.IGNORECASE))


def check_business(programme):
    return bool(re.search(r'\b(business|BA)\b', programme, re.IGNORECASE))


def check_bioinformatics(programme):
    return bool(re.search(r'bio', programme, re.IGNORECASE))


def clean_programmes(df):
    for idx, programme in df['programme'].items():
        if check_ai(programme):
            df.at[idx, 'programme'] = 'Artificial Intelligence'
        elif check_cs(programme):
            df.at[idx, 'programme'] = 'Computer Science'
        elif check_cls(programme):
            df.at[idx, 'programme'] = 'Computational Science'
        elif check_econometrics(programme):
            df.at[idx, 'programme'] = 'Econometrics'
        elif check_finance(programme):
            df.at[idx, 'programme'] = 'Finance'
        elif check_business(programme):
            df.at[idx, 'programme'] = 'Business Analytics'
        elif check_bioinformatics(programme):
            df.at[idx, 'programme'] = 'Bioinformatics'
        else:
            df.at[idx, 'programme'] = 'Other'
            # print(df.at[idx, 'programme'])
    return df

In [ ]:
def convert_course_experience(df):
    for column in ['machine_learning', 'information_retrieval', 'statistics', 'databases']:
        for idx, value in df[column].items():
            if value in ['yes', '1', 'mu', 'ja']:
                df.at[idx, column] = 'yes'
            elif value in ['no', '0', 'sigma', 'nee']:
                df.at[idx, column] = 'no'
    return df

In [ ]:
def convert_gender(df):
    for idx, value in df['gender'].items():
        if value != 'male' and value != 'female':
            df.at[idx, 'gender'] = 'other'
    return df

In [ ]:
def convert_birthdays(df):
    for idx, value in df['birthday'].items():
        value = str(value)
        if re.match(r'^\d{2}-\d{2}-\d{4}$', value):
            # dd-mm-yyyy
            df.at[idx, 'birthday'] = value.replace('-', '/')
        elif re.match(r'^\d{2}-\d{2}-\d{2}$', value):
            # mm-dd-yy
            mm, dd, yy = value.split('-')
            century = '20' if int(yy) <= 25 else '19'
            formatted_value = f"{dd}/{mm}/{century}{yy}"
            df.at[idx, 'birthday'] = formatted_value
        elif re.match(r'^\d{8}$', value):
            # ddmmyyyy
            if float(value[4:]) > 1925:
                formatted_value = f"{value[:2]}/{value[2:4]}/{value[4:]}"
                df.at[idx, 'birthday'] = formatted_value
        elif re.match(r'^\d{2}\.\d{2}\.\d{4}$', value):
            # dd.mm.yyyy
            df.at[idx, 'birthday'] = value.replace('.', '/')
        elif re.match(r'^\d{4}\.\d{2}\.\d{2}$', value):
            # yyyy.mm.dd
            yyyy, mm, dd = value.split('.')
            formatted_value = f"{dd}/{mm}/{yyyy}"
            df.at[idx, 'birthday'] = formatted_value
        elif re.match(r'^\d{4}-\d{2}-\d{2}$', value):
            # yyyy-mm-dd
            yyyy, mm, dd = value.split('-')
            formatted_value = f"{dd}/{mm}/{yyyy}"
            df.at[idx, 'birthday'] = formatted_value
        elif re.match(r'^\d{2} \d{2} \d{4}$', value):
            # dd mm yyyy
            df.at[idx, 'birthday'] = value.replace(' ', '/')
    return df

In [ ]:
def remove_non_numeric(x):
    x = str(x)
    x = re.sub(r'[^0-9.,E+\-]', '', x)
    if '-' in x[1:]:
        x = x.split('-', 1)[0]
    if x.count('E') > 1:
        x = x.replace('E', '')
    
    return x

In [ ]:
def remove_non_numeric_in_column(df, column):
    df[column] = df[column].apply(
        lambda x: remove_non_numeric(x)
    )
    return df

In [ ]:
def clean_and_convert_student_estimate(df):
    df = remove_non_numeric_in_column(df, 'student_estimate')
    df['student_estimate'] = df['student_estimate'].replace('', np.nan)
    df['student_estimate'] = df['student_estimate'].astype(float)
    return df

In [ ]:
def clean_and_convert_stress(df):
    df['stress'] = df['stress'].astype(str)
    df['stress'] = df['stress'].apply(lambda x: re.sub(',', '.', x))
    df = remove_non_numeric_in_column(df, 'stress')
    df['stress'] = df['stress'].replace('', np.nan)
    df['stress'] = df['stress'].astype(float)
    return df

In [ ]:
def clean_and_convert_sports(df):
    df['sports'] = df['sports'].astype(str)
    for idx, value in df['sports'].items():
        if value == 'zero':
            # the one case that had value 'zero'
            df.at[idx, 'sports'] = str(0)
        elif bool(re.search(r'-', value)):
            # range a-b
            lower, higher = value.split('-')
            df.at[idx, 'sports'] = lower
        elif bool(re.search(r'[.,]', value)):
            # decimal (question asked for whole hours)
            integral, _ = re.split(r'[.,]', value, maxsplit=1)
            df.at[idx, 'sports'] = integral
    df = remove_non_numeric_in_column(df, 'sports')
    df['sports'] = df['sports'].replace('', np.nan)
    df['sports'] = df['sports'].astype(float)
    return df

In [ ]:
def clean_and_convert_random_number(df):
    df['random_number'] = df['random_number'].astype(str)
    # international notation '.' = decimal, ',' = exponent of 10^3
    df['random_number'] = df['random_number'].apply(
        lambda x: re.sub(r'\.', '', x) if x.count('.') > 1 else x
    )
    df['random_number'] = df['random_number'].apply(
        lambda x: re.sub(r',', '.', x) if x.count(',') == 1 else x
    )
    df = remove_non_numeric_in_column(df, 'random_number')
    df['random_number'] = df['random_number'].replace('', np.nan)
    df['random_number'] = df['random_number'].astype(float)
    return df

In [ ]:
def convert_to_military_time(time_string):
    formats = [
        "%H:%M", "%Hh%M", "%Hu%M", "%H.%M", "%I.%M", "%H-%M",
        "%H", "%I", "%H%M", "%I%M", "%I%p", "%H%p",
        "%H:%M %p", "%I:%M %p", "%H %p", "%I %p",
        "%H:%M%p", "%I:%M%p", "%I.%M%p"
    ]
    
    for format in formats:
        try:
            dt = datetime.strptime(time_string.strip().lower(), format)
            # Round minutes
            minute = dt.minute
            if minute < 15:
                rounded_minute = 0
            elif minute < 45:
                rounded_minute = 30
            else:
                dt += timedelta(hours=1)
                rounded_minute = 0
            dt = dt.replace(minute=rounded_minute, second=0, microsecond=0)
            if 9 <= dt.hour <= 12:
                dt = dt.replace(hour=(dt.hour + 12) % 24)
            return dt.strftime("%H:%M")
        except ValueError:
            continue

    print(f"Could not parse time_string: {time_string}")
    return time_string

In [ ]:
def convert_bedtime(df):
    for idx, value in df['bedtime'].items():
        df.at[idx, 'bedtime'] = convert_to_military_time(value)
    return df

In [ ]:
def clean_and_save(df):
    df.drop(df.tail(1).index,inplace=True)
    df = clean_programmes(df)
    df = convert_course_experience(df)
    df = convert_birthdays(df)
    df = clean_and_convert_student_estimate(df)
    df = clean_and_convert_stress(df)
    df = clean_and_convert_sports(df)
    df = clean_and_convert_random_number(df)
    df = convert_bedtime(df)
    save_dataframe_to_file(df, 'DMT_Data.csv')

In [ ]:
clean_and_save(import_raw_data())

After data formatting and saving to DMT_Data.csv, some birthdays were manually adjusted to fit the format dd/mm/yyyy, and some bedtimes to hh:mm. Furthermore, rows 174, 240 were removed for being invalid. The result was saved to DMT_Data_Manual.csv.

### Combining Data

In [ ]:
def calculate_age(df):
    df['birthday'] = pd.to_datetime(df['birthday'], format='%d/%m/%Y', errors='coerce')
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    
    df['age'] = df.apply(
        lambda row: (row['timestamp'].year - row['birthday'].year) 
                    - ((row['timestamp'].month, row['timestamp'].day) < 
                       (row['birthday'].month, row['birthday'].day)),
        axis=1
    ).astype(float)
    return df

In [ ]:
df = import_data('DMT_Data_Manual.csv')
save_dataframe_to_file(calculate_age(df), 'DMT_Data_Age.csv')

### Removing Invalid Data
Removal of data based on values outside of a realistic/predetermined range.

In [ ]:
def remove_invalid_ages(df):
    df['age'] = df['age'].where((df['age'] >= 16) & (df['age'] <= 80), np.nan)
    return df

In [ ]:
def remove_invalid_student_estimates(df):
    df['student_estimate'] = df['student_estimate'].where((df['student_estimate'] >= 50) & (df['student_estimate'] <= 800), np.nan)
    return df

In [ ]:
def remove_invalid_stress(df):
    df['stress'] = df['stress'].where((df['stress'] >= 0) & (df['stress'] <= 100), np.nan)
    return df

In [ ]:
def remove_invalid_sports(df):
    df['sports'] = df['sports'].where((df['sports'] >= 0) & (df['sports'] <= 40), np.nan)
    return df

In [ ]:
def remove_invalid_bedtimes(df):
    pattern = r'^\d{2}:\d{2}$'
    df['bedtime'] = df['bedtime'].astype(str).apply(
        lambda x: x if re.match(pattern, x) else np.nan
    )
    return df

In [ ]:
def remove_invalid_data(df):
    df = remove_invalid_ages(df)
    df = remove_invalid_student_estimates(df)
    df = remove_invalid_stress(df)
    df = remove_invalid_sports(df)
    df = remove_invalid_bedtimes(df)
    save_dataframe_to_file(df, 'DMT_Data_Clean.csv')

In [ ]:
remove_invalid_data(import_data('DMT_Data_Age.csv'))

### Imputation
Replacing missing values
#### Method 1: Replace with Median

In [ ]:
def median_bedtime(bedtimes):
    bedtimes_DT = pd.to_datetime(bedtimes, format='%H:%M', errors='coerce')
    bedtimes_seconds = (bedtimes_DT.dt.hour * 3600 + bedtimes_DT.dt.minute * 60)
    # Center around midnight s.t. 22:00 becomes -02:00
    bedtimes_centered = bedtimes_seconds.apply(lambda s: s - (24 * 3600) if s > (12 * 3600) else s)
    median_seconds = bedtimes_centered.median()
    median_time = (datetime.min + timedelta(seconds=median_seconds)).strftime('%H:%M')
    return median_time

In [ ]:
def impute_median(df):
    cols = ['student_estimate', 'stress', 'sports', 'bedtime', 'age']
    for col in cols:
        if col == 'bedtime':
            median = median_bedtime(df[col])
        else:
            median = df[col].median()
        print('median for col ' + col + ': ' + str(median))
        df[col] = df[col].fillna(median)
    return df

In [ ]:
save_dataframe_to_file(impute_median(import_data('DMT_Data_Clean.csv')), 'DMT_Data_Median.csv')

### Transforming data

#### Numerical Classifier

In [ ]:
def one_hot_encode(df, columns):
    return pd.get_dummies(df, columns=columns)

In [ ]:
def normalize(df, columns):
    for col in columns:
        min_value = min(df[col])
        max_value = max(df[col])
        df[col] = (df[col]-min_value)/(max_value-min_value)
    return df

In [ ]:
def bedtime_to_mins_past_three(df):
    bedtimes = pd.to_datetime(df['bedtime'], format='%H:%M').dt.time
    bedtimes_mins = bedtimes.apply(lambda t: t.hour * 60 + t.minute)
    df['bedtime'] = (bedtimes_mins - 900) % 1440
    return df

In [ ]:
def encode_numerically(df):
    categorical = ['programme', 'machine_learning', 'information_retrieval', 'statistics', 'databases', 'gender', 'chatgpt']
    numerical = ['student_estimate', 'stress', 'sports', 'bedtime', 'age']
    df = bedtime_to_mins_past_three(df)
    df = one_hot_encode(df, categorical)
    df = normalize(df, numerical)
    return df

In [ ]:
df = encode_numerically(import_data('DMT_Data_Clean.csv'))
save_dataframe_to_file(df, 'DMT_Data_Numerical.csv')

#### Categorical Classifier

In [ ]:
def encode_programme(df):
    programmes = df['programme']
    programme_mapping = {program: idx for idx, program in enumerate(programmes.unique())}
    # make 'Other' the last entry
    last_integer = len(programme_mapping) - 1
    last_programme = [programme for programme, value in programme_mapping.items() if value == last_integer][0]
    other_integer = programme_mapping['Other']
    programme_mapping['Other'] = last_integer
    programme_mapping[last_programme] = other_integer
    df['programme'] = df['programme'].map(programme_mapping)
    print("Programmes mapped:", programme_mapping)
    return df

In [ ]:
def encode_student_estimate(df):
    # low = 0-300, Medium = 301-450, High = 451+
    bins = [0, 300, 450, float('inf')]
    labels = ['low', 'medium', 'high']
    df['student_estimate'] = pd.cut(df['student_estimate'], bins=bins, labels=labels, right=True)
    return df

In [ ]:
def encode_stress(df):
    # Low = 0-33, Medium = 34-66, High = 67-100
    bins = [0, 33, 66, 100]
    labels = ['low', 'medium', 'high']
    df['stress'] = pd.cut(df['stress'], bins=bins, labels=labels, right=True)
    return df

In [ ]:
def encode_sports(df):
    # Low = 0-2, Medium = 3-6, High = 7+
    bins = [0, 2, 6, float('inf')]
    labels = ['low', 'medium', 'high']
    df['sports'] = pd.cut(df['sports'], bins=bins, labels=labels, right=True)
    return df

In [ ]:
def encode_bedtime(df):
    df = bedtime_to_mins_past_three(df)
    # Early = before 10pm, Middle = 10:01pm-1am, Late = past 1:01am
    bins = [0, 420, 600, float('inf')]
    labels = ['early', 'middle', 'late']
    df['bedtime'] = pd.cut(df['bedtime'], bins=bins, labels=labels, right=True)
    return df

In [ ]:
def encode_age(df):
    # Young = 0-21, Middle = 22-25, Old = 26+
    bins = [0, 21, 25, float('inf')]
    labels = ['young', 'middle', 'old']
    df['age'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)
    return df

In [ ]:
def encode_categorically(df):
    # df = encode_programme(df)
    df = encode_student_estimate(df)
    df = encode_stress(df)
    df = encode_sports(df)
    df = encode_bedtime(df)
    df = encode_age(df)
    save_dataframe_to_file(df, 'DMT_Data_Categorical.csv')

In [ ]:
df = import_data('DMT_Data_Clean.csv')
encode_categorically(df)

## Plotting

In [ ]:
def plot_bedtimes(df):
    bedtimes_DT = pd.to_datetime(df['bedtime'], format='%H:%M', errors='coerce')
    bedtimes_seconds = bedtimes_DT.dt.hour * 3600 + bedtimes_DT.dt.minute * 60

    # Center bedtimes around midnight
    half_day = 43200  # 12 hours in seconds
    centered_bedtimes = bedtimes_seconds.apply(lambda x: x - 86400 if x > half_day else x)
    
    # Sort centered bedtimes
    sorted_bedtimes = np.sort(centered_bedtimes)

    # Create the histogram from sorted values
    counts = pd.Series(sorted_bedtimes).value_counts().sort_index()

    plt.figure(figsize=(10, 6))
    counts.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title(f"Histogram of bedtimes")
    plt.xlabel("bedtime")
    plt.ylabel("Count")
    tick_labels = [str(datetime.utcfromtimestamp(x).strftime('%H:%M')) for x in counts.index]  # Convert seconds to hh:mm
    plt.xticks(ticks=np.arange(len(tick_labels)), labels=tick_labels, rotation=90)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_stress(df):
    plt.figure(figsize=(10, 6))
    df['stress'].plot(kind='hist', bins=10, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title("Histogram of Stress")
    plt.xlabel("Stress")
    plt.ylabel("Count")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_dist(df, column):
    plt.figure(figsize=(10, 6))
    df[column].plot(kind='hist', bins=10, color='skyblue', edgecolor='black', alpha=0.7)
    
    plt.title(f"Histogram of '{column}'")
    plt.xlabel(column)
    plt.ylabel("Count")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_numerical_imputations():
    cols = ['student_estimate', 'stress', 'sports', 'bedtime', 'age']
    df_med = import_data('DMT_Data_Median.csv')
    df_na = import_data('DMT_Data_Clean.csv')
    for col in cols:
        if col == 'bedtime':
            plot_bedtimes(df_med)
            plot_bedtimes(df_na)
        else:
            plot_dist(df_med, col)
            plot_dist(df_na, col)

In [ ]:
plot_numerical_imputations()